# Jaqal to IR

In this tutorial we will examine Jaqal. [Jaqal](https://www.sandia.gov/quantum/quantum-information-sciences/projects/qscout-jaqal/) (Just Another Quantum Assembly Language) is the QASM built by Sandia Labs for the [QSCOUT](https://www.sandia.gov/quantum/quantum-information-sciences/projects/qscout/) trapped-ion backend.

## JaqalPaq

We will primarily be using [JaqalPaq](https://gitlab.com/jaqal/jaqalpaq), the python package for Jaqal meta-programming. For details about JaqalPaq installation, see their [guide](https://gitlab.com/jaqal/jaqalpaq).

## Basic Usage Example

We've provided the `Sxx_circuit.jaqal` file in this repository: to confirm that your installation is valid, run the following code cell. You should see an idealized result array from the noiseless emulator: 50% |00>, 50% |11>

In [1]:
import jaqalpaq
from jaqalpaq.parser import parse_jaqal_file
from jaqalpaq.emulator import run_jaqal_circuit
from jaqalpaq.generator import generate_jaqal_program

JaqalCircuitObject = parse_jaqal_file("Sxx_circuit.jaqal")
JaqalCircuitResults = run_jaqal_circuit(JaqalCircuitObject)
print(f"Probabilities: {JaqalCircuitResults.subcircuits[0].probability_by_str}")
JaqalProgram = generate_jaqal_program(JaqalCircuitObject)

c:\Users\piegu\AppData\Local\Programs\Python\Python38\lib\site-packages\pygsti\objects\replib\__init__.py:29: UserWarning: 
An optimized Cython-based implementation of `pygsti.objects.replib` is available as
an extension, but couldn't be imported. This might happen if the
extension has not been built. `pip install cython`, then reinstall
pyGSTi to build Cython extensions. Alternatively, setting the
environment variable `PYGSTI_NO_CYTHON_WARNING` will suppress this
message.

  _warnings.warn(warn_msg)
c:\Users\piegu\AppData\Local\Programs\Python\Python38\lib\site-packages\pygsti\objects\opcalc\__init__.py:31: UserWarning: 
An optimized Cython-based implementation of `pygsti.objects.opcalc` is available as
an extension, but couldn't be imported. This might happen if the
extension has not been built. `pip install cython`, then reinstall
pyGSTi to build Cython extensions. Alternatively, setting the
environment variable `PYGSTI_NO_CYTHON_WARNING` will suppress this
message.

  _warnings.war

Probabilities: OrderedDict([('00', 0.5000000000000001), ('10', 0.0), ('01', 0.0), ('11', 0.5000000000000001)])


## Translating to Jaqal

Jaqal has built in capability to take circuits from a variety of widly used languages (Qiskit, Quil, pytket, Cirq) and turn them into Jaqal code which can run on QSCOUT. We will walk through the transpilation of our Bernstein-Vazirani circuit from Qiskit into `bernstein_vazirani.jaqal`.

In [3]:
from qiskit import QuantumCircuit


s = '011'   # the hidden binary string
n = 3 # number of bits used to represent s


# We need a circuit with n qubits, plus one auxiliary qubit
# We also need n classical bits to write the output to
qc = QuantumCircuit(n+1, n)

# Put auxiliary in the minus state |->
# We leave the other n qubits in just |0>
qc.h(n)
qc.z(n)

# Apply Hadamard gates to each of the n "main" qubits (excluding the auxiliary)
# before querying the oracle
for i in range(n):
    qc.h(i)
    
# Apply barrier
qc.barrier()

# Apply the inner-product oracle
s = s[::-1] # reverse s to fit qiskit's qubit ordering
for q in range(n):
    if s[q] == '0':
        qc.i(q)
    else:
        qc.cx(q, n)
        
# Apply barrier
qc.barrier()

# Apply Hadamard gates after querying the oracle
for i in range(n):
    qc.h(i)


# Measurement
for i in range(n):
    qc.measure(i, i)

qc.draw()

┌───┐      ░                 ░ ┌───┐┌─┐      
q_0: ┤ H ├──────░───■─────────────░─┤ H ├┤M├──────
     ├───┤      ░   │             ░ ├───┤└╥┘┌─┐   
q_1: ┤ H ├──────░───┼─────────■───░─┤ H ├─╫─┤M├───
     ├───┤      ░   │  ┌───┐  │   ░ ├───┤ ║ └╥┘┌─┐
q_2: ┤ H ├──────░───┼──┤ I ├──┼───░─┤ H ├─╫──╫─┤M├
     ├───┤┌───┐ ░ ┌─┴─┐└───┘┌─┴─┐ ░ └───┘ ║  ║ └╥┘
q_3: ┤ H ├┤ Z ├─░─┤ X ├─────┤ X ├─░───────╫──╫──╫─
     └───┘└───┘ ░ └───┘     └───┘ ░       ║  ║  ║ 
c: 3/═════════════════════════════════════╩══╩══╩═
                                          0  1  2

## Translation to Jaqal

Translation from Qiskit to Jaqal occurs in two steps. First, the above circuit must be unrolled to QSCOUT's native gate set, composed of ion compatible instructions. Then, we can directly translate that circuit object into a Jaqal circuit object.

In [8]:
from qiskit.transpiler import PassManager
from jaqalpaq.transpilers.qiskit import ion_pass_manager, jaqal_circuit_from_qiskit_circuit

# ion_pass_manager provides a qiskit PassManager object which is supposed to 
# unroll a qiskit circuit to Jaqal gates, however it struggles
# with Qiskit 'I' gate
ion_qc = ion_pass_manager().run(qc)


QiskitError: "Cannot unroll the circuit to the given basis, dict_keys(['jaqalr', 'sx', 'sxdg', 'sy', 'sydg', 's', 'sdg', 'x', 'y', 'z', 'rz', 'jaqalms']). Instruction id not found in equivalence library and no rule found to expand."

In [10]:
# In order to deal with I, we need to add a relation to the Equivalence Library
# of a pass called 'Unroll Custom Definitions'. To get at that pass:
pm = ion_pass_manager()
passes_array_dict_array = pm.passes()
passes_array_dict = passes_array_dict_array[0]
passes_array = passes_array_dict['passes']
unroll_custom_defs = passes_array[0]
print(dir(unroll_custom_defs))



['__call__', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_basis_gates', '_equiv_lib', '_hash', 'is_analysis_pass', 'is_transformation_pass', 'name', 'preserves', 'property_set', 'requires', 'run']


In [36]:
# Now, we get the equivalence library and add the relation
# I_Px (Jaqal Identity gate with same duration as Px) = I
equiv_lib = unroll_custom_defs._equiv_lib
#print(dir(equiv_lib))
#print(equiv_lib.__dict__.items())
print(equiv_lib.__dict__['_base'].__dict__.items())
# equiv_lib.add_equivalence()


#pm._equiv_lib.append()
#jaq_compat = pm.run(qc)

#jaqc = jaqal_circuit_from_qiskit_circuit(jaq_compat) #qc, qiskit_to_jaq_map)
#print(generate_jaqal_program(jaqc))

dict_items([('_base', None), ('_map', {Key(name='h', num_qubits=1): Entry(search_base=True, equivalences=[Equivalence(params=[], circuit=<qiskit.circuit.quantumcircuit.QuantumCircuit object at 0x0000020DCBA69CA0>), Equivalence(params=[], circuit=<qiskit.circuit.quantumcircuit.QuantumCircuit object at 0x0000020DCD07CEB0>), Equivalence(params=[], circuit=<qiskit.circuit.quantumcircuit.QuantumCircuit object at 0x0000020DCD08C070>)]), Key(name='ch', num_qubits=2): Entry(search_base=True, equivalences=[Equivalence(params=[], circuit=<qiskit.circuit.quantumcircuit.QuantumCircuit object at 0x0000020DA70581F0>)]), Key(name='p', num_qubits=1): Entry(search_base=True, equivalences=[Equivalence(params=[Parameter(theta)], circuit=<qiskit.circuit.quantumcircuit.QuantumCircuit object at 0x0000020DCB9710A0>), Equivalence(params=[Parameter(theta)], circuit=<qiskit.circuit.quantumcircuit.QuantumCircuit object at 0x0000020DCCEAB580>)]), Key(name='cp', num_qubits=2): Entry(search_base=True, equivalences=